### MERGE ALL DATA FROM NOTEBOOKS 1 THRU 4

In [ ]:
# Libraries.
import pandas as pd
import numpy as np
from pathlib import Path

In [ ]:
# Load data.
heat = pd.read_csv("data/heat/jfk_daily_heat_2025.csv", parse_dates = ["DATE"])
panel = pd.read_csv("data/panel/nyc_311_tract_day_2025.csv", parse_dates = ["date"])
acs = pd.read_csv("data/acs/acs_socioeconomic_tract_2022.csv", dtype = {"GEOID":str})
env = pd.read_csv("data/raster/environmental_raster_tract_2025.csv", dtype = {"GEOID":str})

In [ ]:
# Merge heat.
df = panel.merge(heat, left_on = "date", right_on = "DATE", how = "left")
df.drop(columns = ["DATE"], inplace = True)

In [ ]:
# Merge ACS.
df = df.merge(acs, on = "GEOID", how = "left")

In [ ]:
# Merge environmental variables.
df = df.merge(env, on = "GEOID", how = "left")

In [ ]:
# Derived variables.
df["log_total_calls"] = np.log(df["total_calls"].replace(0, 1))

df["poverty_rate_c"] = df["poverty_rate"] - df["poverty_rate"].mean()
df["medhhinc_c"] = df["medhhinc"] - df["medhhinc"].mean()
df["ndvi_c"] = df["ndvi_mean"] - df["ndvi_mean"].mean()
df["impervious_c"] = df["impervious_mean"] - df["impervious_mean"].mean()

df["extreme_x_poverty"] = df["EXTREME_HEAT"] * df["poverty_rate_c"]
df["extreme_x_impervious"] = df["EXTREME_HEAT"] * df["impervious_c"]

df["dow"] = df["date"].dt.dayofweek
df["doy"] = df["date"].dt.dayofyear

In [ ]:
# Save.
out_file = Path("data/model/nyc_tract_daily_qol_heat_2025_final.csv")
out_file.parent.mkdir(parents = True, exist_ok = True)

df.to_csv(out_file, index = False)
print("Saved:", out_file)

df.head()